In [1]:
%load_ext autoreload
%autoreload

import os
import sys
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.apps import apps
from django_currentuser.middleware import get_current_user

# MaskBase

In [2]:
#m.MaskBase.objects.all().delete()
m.MaskBase.objects.all()

<QuerySet []>

In [3]:
dir_masks = Path(parentdir) / 'getch/static/materials/masks'

In [4]:
for d in dir_masks.glob('*'):
    if d.name != 'archives':
        _type = d.name.upper()
        
        for f in d.rglob('*.*'):
            cat = f.parts[-2]
            
            if cat != 'raw':
                maskbase = m.MaskBase.objects.create(type=_type, category=cat)
                maskbase.pix.save(f.name, ContentFile(f.read_bytes()), save=True)

# MaskSerializer

In [7]:
class EyeMaskSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.EyeMask
        fields = ['id', 'maskbase', 'top', 'left', 'width', 'height']
        read_only_fields = ['id']
        
class MouthMaskSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.MouthMask
        fields = ['id', 'maskbase', 'top', 'left', 'width', 'height']
        read_only_fields = ['id']        
        
EyeMaskSerializer()

EyeMaskSerializer():
    id = IntegerField(read_only=True)
    maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all(), required=False)
    top = FloatField(required=False)
    left = FloatField(required=False)
    width = FloatField(required=False)
    height = FloatField(required=False)

## Serializing

In [8]:
eyemask = m.EyeMask.objects.first()
data = EyeMaskSerializer(eyemask).data; data

{'id': 1, 'maskbase': 1, 'top': 0.0, 'left': 0.0, 'width': 100.0, 'height': 20.0}

## Deserializing
### 1. Create
* serializer로 creation 하지 않는다
* profile 생성할때 mask(1-1관계)는 default로 무조건 같이 생성되며, 이때 Mask.objects.create()를 쓴다

In [2]:
eyemask = m.EyeMask.objects.create(); eyemask;

In [9]:
# serializer = MaskSerializer(data={'maskbase':15, 'top':10, 'left':0, 'width':50, 'height':50})
# serializer.is_valid()
# mask = serializer.save(); mask

### 2. Update

In [12]:
serializer = EyeMaskSerializer(eyemask, data={'maskbase':15, 'top':10, 'left':0, 'width':50, 'height':50})
serializer.is_valid()
eyemask = serializer.save(); eyemask

<EyeMask: EYE | cartoon_m | 2020-08-11_095158.942119SIDEB_EYEMASK_M_CARTOON_05_onepunman.jpg | T=10.0, L=0.0, W=50.0, H=50.0>

# ProfileSerializer

In [8]:
class ProfileSerializer(serializers.ModelSerializer):
    eye_mask = MaskSerializer(required=False)
    mouth_mask = MaskSerializer(required=False)

    class Meta:
        model = m.Profile
        fields = ['id', 'type', 'pix', 'image', 'text', 'eye_mask', 'mouth_mask']
        read_only_fields = ['id']
        
ProfileSerializer()

ProfileSerializer():
    id = IntegerField(read_only=True)
    type = ChoiceField(choices=(('IMAGE', 'image'), ('TEXT', 'text')), required=False)
    pix = ImageField(max_length=500)
    image = ImageField(allow_null=True, max_length=500, required=False)
    text = CharField(allow_blank=True, allow_null=True, max_length=10, required=False)
    eye_mask = MaskSerializer(required=False):
        id = IntegerField(read_only=True)
        maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
        top = FloatField(required=False)
        left = FloatField(required=False)
        width = FloatField(required=False)
        height = FloatField(required=False)
    mouth_mask = MaskSerializer(required=False):
        id = IntegerField(read_only=True)
        maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
        top = FloatField(required=False)
        left = FloatField(required=False)
        width = FloatField(required=False)
        height = FloatField(requ

## Serializing

In [14]:
profile = m.Profile.objects.last()
data = ProfileSerializer(profile).data; data

{'id': 8, 'type': 'IMAGE', 'pix': 'https://storage.googleapis.com/sideb-proejct.appspot.com/user/gem763%40gmail.com/2020-08-04_134305.193770__killyou.gif?Expires=1597127094&GoogleAccessId=sideb-serviceaccount%40sideb-proejct.iam.gserviceaccount.com&Signature=R4AlbT3eCphmIbVR1Vc4EBaD779rlAeqCHACK2CmHLSTiK6wWYM8tth8n4kmRhnt3ndRv%2Bn78N%2B304ezDG%2BiimpiYR3SOtSzXT%2FLttzRomu6d94coa1WRjgxgNLu%2BJ%2F7EDWnX8RagxLaAxtPCY%2BKf6AN59VN6%2BEY0YtARjcvNoSR2upvF0genDVNKJEZzfosC3AVZZeq%2BoB0xwWiR9tCVlvoshYgZIiZ%2B5QO%2B4TaKPJs55DkYZp%2BGHPWPDiW4Bi1XgYCtqOFMSy%2FKyeGa8sN25M72JwewucrbD7YarkXX3EZNxdhfnqSzqW3yv6D0BU6pTclPF1OhyYyebQO9iTZvw%3D%3D', 'image': None, 'text': None, 'eye_mask': OrderedDict([('id', 2), ('maskbase', 14), ('top', 0.0), ('left', 0.0), ('width', 100.0), ('height', 20.0)]), 'mouth_mask': None}

## Deserializing
### 1. create

In [15]:
pix = m.PostVoteOX.objects.all()[0].pix; pix

<ImageFieldFile: post/2020/7/22/sejin76@nate.com/2020-07-22_195937.085170__AB_anorak_B.jpg>

In [17]:
serializer = ProfileSerializer(data={'type':'IMAGE', 'text':'TESTTEST', 'pix':None, 'image':None, 'eye_mask':None, 'mouth_mask':None})
serializer.is_valid()
# profile = serializer.save(); profile

False

In [19]:
serializer.validated_data

{}

In [2]:
profile = m.Profile.objects.create()

In [4]:
profile.pix

<ImageFieldFile: material/profile.png>

In [2]:
user = m.User.objects.get(pk=1)
m.Boo.objects.create(user=user)

<Boo: sideb | sideb@sideb.com>

# PostSerializer

In [2]:
class PostVoteOXSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.PostVoteOX
        fields = ['id', 'boo', 'text', 'pix']
        read_only_fields = ['id']
        
class PostVoteABSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.PostVoteAB
        fields = ['id', 'boo', 'text', 'pix_a', 'pix_b']
        read_only_fields = ['id']
    
class PostSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.Post
        fields = '__all__'
        
    def to_representation(self, instance):
        if isinstance(instance, m.PostVoteOX):
            return PostVoteOXSerializer(instance=instance).data
        
        elif isinstance(instance, m.PostVoteAB):
            return PostVoteABSerializer(instance=instance).data
        
    def create(self, validated_data):
        _type = validated_data.pop('type')
        
        postmodel = apps.get_model(app_label='getch', model_name=_type)
        return postmodel.objects.create(**validated_data)       
    
            
PostVoteOXSerializer()
PostSerializer()

PostSerializer():
    id = IntegerField(read_only=True)
    text = CharField(allow_blank=True, allow_null=True, max_length=500, required=False, style={'base_template': 'textarea.html'})
    created_at = DateTimeField(read_only=True)
    boo = PrimaryKeyRelatedField(allow_null=True, queryset=Boo.objects.all(), required=False)

## Serializing

In [3]:
posts = m.Post.objects.select_subclasses()
data = PostSerializer(posts, many=True).data; data[6]

{'id': 12, 'boo': 14, 'text': '구매예정', 'pix': 'https://storage.googleapis.com/sideb-proejct.appspot.com/post/2020/8/13/sejin76%40nate.com/2020-08-13_172307.570023__shoeA.jpg?Expires=1597909838&GoogleAccessId=sideb-serviceaccount%40sideb-proejct.iam.gserviceaccount.com&Signature=VW2svxky2v%2BfnYyoQbwtxqjS5B3o%2FrgV8BKkhOFtLwSGV3tPv4LQ%2BSPVdVsNhj4td2dYINlXcili5Rjx1xZtruaQN09YMAy127c7uEec7lekQj1Wbgwc6YxmV4UYS%2FIoMOSSykY54W762%2BUECi48PpKeg0nym7O47D%2FOrhb3fA7FKb5oGeWQItSbbGk%2BVWHZ5rWXNghIBRv2Uqzat9FdIpfLj3YCpa8TIIVUVOdNGhAJnDWCQwDPT9vCQDX9tqTtdPU%2ByvOOMPzBb2x6tKsUbUyqzCgqcWZi0OtpGYzDZyMw2KUirou18BezVI8wPTVXcsb3emgqDzcrlqzwWETPDg%3D%3D'}

In [4]:
posts[0].pix

<ImageFieldFile: post/2020/8/12/sideb@sideb.com/2020-08-12_181653.835216__sideb_logo.jpg>

## Deserializing
### 1. Create

In [9]:
ser = PostVoteOXSerializer(data={'boo':15, 'text':'이건멀까나#고군분투', 'pix':posts[0].pix})
ser.is_valid()
post = ser.save(); post

<PostVoteOX: OX | >

In [5]:
ser = PostSerializer(data={'text':'이건멀까나#고군분투', 'pix':posts[0].pix})
ser.is_valid()
# post = ser.save(); post

True

In [10]:
ser.initial_data

{'type': 'postvoteox',
 'text': '이건멀까나#고군분투',
 'pix': <ImageFieldFile: post/2020/8/12/sideb@sideb.com/2020-08-12_181653.835216__sideb_logo.jpg>}

### 2. Update

In [11]:
ser = PostVoteOXSerializer(post, data={'boo':15, 'text':'자나깨나뽕#고군분투'}, partial=True)
ser.is_valid()
post = ser.save(); post

<PostVoteOX: OX | >

In [12]:
ser.validated_data

OrderedDict([('boo', <Boo: 모지리왕# | sejin76@nate.com>), ('text', '자나깨나뽕#고군분투')])